# Security Sentinel

### Audit and harden your code in one sweep

Choose a contract to inspect a snippet for security issues:

- Scan for vulnerabilities
- Draft threat intelligence notes
- Suggest secure fixes
- Craft exploit-focused tests

In [ ]:
import os
import logging
from enum import StrEnum
from getpass import getpass

import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
logging.basicConfig(level=logging.WARNING)

logger = logging.getLogger('sentinel')
logger.setLevel(logging.DEBUG)

In [ ]:
def get_secret_in_google_colab(env_name: str) -> str:
    try:
      from google.colab import userdata
      return userdata.get(env_name)
    except Exception:
      return ''


def get_secret(env_name: str) -> str:
    key = os.environ.get(env_name) or get_secret_in_google_colab(env_name)

    if not key:
        key = getpass(f'Enter {env_name}:').strip()

    if key:
        logger.info(f'✅ {env_name} provided')
    else:
        logger.warning(f'❌ {env_name} not provided')
    return key.strip()

In [ ]:
class Provider(StrEnum):
    OLLAMA = 'Ollama'

clients: dict[Provider, OpenAI] = {}

clients[Provider.OLLAMA] = OpenAI(base_url='http://localhost:11434/v1')

model = 'llama3.2:latest'
client = clients.get(Provider.OLLAMA)
if not client:
    raise Exception('No client found')

In [ ]:
class Task(StrEnum):
  SCAN = 'Scan'
  REPORT = 'Threat Report'
  PATCH = 'Patch'
  TEST = 'Exploit Test'


def perform_tasks(tasks, code):
  logger.info(f'Performing tasks: {tasks}')

  steps = []
  if Task.SCAN in tasks:
    steps.append('Scan the snippet for security weaknesses and name them clearly.')
  if Task.REPORT in tasks:
    steps.append('Produce a concise threat report that explains impact, likelihood, and affected components.')
  if Task.PATCH in tasks:
    steps.append('Propose hardened code that mitigates the identified risks without changing intent.')
  if Task.TEST in tasks:
    steps.append('Design exploit-style tests or probing steps that would validate the vulnerability.')

  task_list = '- ' + '\n- '.join(steps) if steps else 'No security directive selected.'
  system_prompt = f"""
  You are a seasoned application security engineer who recognises languages instantly and
  maps weaknesses to common vulnerability classes.
  Only rewrite code when instructed to patch it.

  Your tasks:
  {task_list}
  """
  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": f'Code: \n{code}'}
  ]
  response =  client.chat.completions.create(
      model=model,
      messages=messages
  )

  content = response.choices[0].message.content

  return content

In [ ]:
def get_examples() -> tuple[list[any], list[str]]:
    python_sql = r'''
    import sqlite3

    def get_user(conn, user_id):
        query = f"SELECT * FROM users WHERE id = {user_id}"
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchone()
    '''

    js_auth = r'''
    app.post('/login', async (req, res) => {
        const token = jwt.decode(req.body.token);
        if (!token) {
            return res.status(401).send('blocked');
        }
        const user = await db.find(token.user);
        res.send(user);
    });
    '''

    go_crypto = r'''
    package main

    import (
        "crypto/sha1"
    )

    func hashPassword(password string) string {
        h := sha1.New()
        h.Write([]byte(password))
        return string(h.Sum(nil))
    }
    '''

    php_upload = r'''
    <?php
    if ($_FILES["file"]["error"] == 0) {
        move_uploaded_file($_FILES["file"]["tmp_name"], "/uploads/" . $_FILES["file"]["name"]);
        echo "done";
    }
    ?>
    '''

    rust_config = r'''
    use std::env;

    fn main() {
        let endpoint = env::var("SERVICE_URL").unwrap();
        println!("Connecting to {}", endpoint);
    }
    '''

    examples = [
        [[Task.SCAN], python_sql, 'python'],
        [[Task.REPORT], js_auth, 'javascript'],
        [[Task.PATCH], go_crypto, 'go'],
        [[Task.TEST], php_upload, 'php'],
        [[Task.SCAN, Task.PATCH, Task.REPORT], rust_config, 'rust']
    ]

    example_labels = [
        'Python: SQL injection review',
        'JavaScript: Token handling report',
        'Go: Strengthen hashing',
        'PHP: Exploit upload path',
        'Rust: Exposure analysis'
    ]

    return examples, example_labels

## UI

In [ ]:
title = 'Security Sentinel'

with gr.Blocks(title=title, theme=gr.themes.Monochrome()) as ui:
    gr.Markdown(f'# {title}')
    gr.Markdown('## Run rapid security sweeps on any snippet.')

    with gr.Row():
      with gr.Column():
        tasks = gr.Dropdown(
            label="Missions",
            choices=[task.value for task in Task],
            value=Task.SCAN,
            multiselect=True,
            interactive=True,
        )
        code_input = gr.Code(
            label='Code Input',
            lines=40,
        )
        code_language = gr.Textbox(visible=False)

      with gr.Column():
        gr.Markdown('## Findings')
        code_output = gr.Markdown('Awaiting report')


    run_btn = gr.Button('Run Audit')

    def set_language(tasks, code, language):
      syntax_highlights = ['python', 'c', 'cpp', 'javascript', 'typescript', 'go', 'rust', 'php']
      logger.debug(f'Tasks: {tasks}, Language: {language}')
      highlight = language if language in syntax_highlights else None

      return tasks, gr.Code(value=code, language=highlight)

    examples, example_labels = get_examples()
    examples = gr.Examples(
        examples=examples,
        example_labels=example_labels,
        examples_per_page=20,
        inputs=[tasks, code_input, code_language],
        outputs=[tasks, code_input],
        run_on_click=True,
        fn=set_language
    )

    run_btn.click(perform_tasks, inputs=[tasks, code_input], outputs=[code_output])

ui.launch(debug=True)